# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 10:53:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2021년 LG Data**

In [5]:
lexicon_2021 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2021_lg.csv')

lexicon_2021.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,LG화학,51910,매일경제,2021010104,[정유·화학] 최악국면 지났지만…석유제품 수요 변수,http://news.mk.co.kr/newsRead.php?no=391&year=...,◆ 신년기획 2021 경제기상도 ◆\r\n\n\n충남에 위치한 현대중공업 대산공장에...,2021-01-04,4,849000,900000,835000,889000,858451,0.078883,1,1,신년 기획 경제 기상도 충남 위치 현대 중공업 대산 공장 직원 업무 열중 모습 사진...


In [6]:
lexicon_2021[lexicon_2021['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2021.dropna(axis=0, inplace=True)
lexicon_2021 = lexicon_2021.reset_index(drop=True)
lexicon_2021.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
neg_lexicon_2021 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_neg_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_neg와의 Cosine Similarity 계산
for x in range(len(lexicon_2021['Tokenization'])):
    news_num = x+1
    for y in range(len(negative)):
        for z in range(len(list(set(lexicon_2021['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2021['Tokenization'][x].split())):
                    if lexicon_2021['Tokenization'][x].split()[w] == list(set(lexicon_2021['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2021['date'][x],
                    'news_num': news_num,
                    'KOSELF_neg_word': negative[y],
                    'news_word': list(set(lexicon_2021['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                neg_lexicon_2021 = neg_lexicon_2021.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2021['date'][x], negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z], ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z])))

***2021-01-04 Cosine Similarity between <둔화> & <하락> : 0.5260781049728394
***2021-01-04 Cosine Similarity between <둔화> & <위축> : 0.6352296471595764
***2021-01-04 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2021-01-04 Cosine Similarity between <둔화> & <침체> : 0.5701860189437866
***2021-01-04 Cosine Similarity between <약화> & <위축> : 0.5172011256217957
***2021-01-04 Cosine Similarity between <침체> & <경기> : 0.5360627174377441
***2021-01-04 Cosine Similarity between <침체> & <위축> : 0.5503976345062256
***2021-01-04 Cosine Similarity between <침체> & <침체> : 0.9999999403953552
***2021-01-04 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2021-01-04 Cosine Similarity between <하락> & <주춤> : 0.5109283328056335
***2021-01-04 Cosine Similarity between <하락> & <상승> : 0.7126480340957642
***2021-01-04 Cosine Similarity between <둔화> & <하락> : 0.5260781049728394
***2021-01-04 Cosine Similarity between <둔화> & <급락> : 0.5491443276405334
***2021-01-04 Cosine Similarity between <하락> & <급락> 

In [11]:
neg_lexicon_2021

,date,news_num,KOSELF_neg_word,news_word,cosine_similarity,frequency
0,2021-01-04,1,둔화,하락,0.526078,2
1,2021-01-04,1,둔화,위축,0.635230,1
2,2021-01-04,1,둔화,증가,0.504186,4
3,2021-01-04,1,둔화,침체,0.570186,2
4,2021-01-04,1,약화,위축,0.517201,1
...,...,...,...,...,...,...
7094,2021-09-16,1511,침체,위축,0.550398,1
7095,2021-09-16,1511,하락,보합,0.617605,2
7096,2021-09-16,1511,하락,증가,0.553474,1
7097,2021-09-16,1511,하락,상승,0.712648,23


In [12]:
neg_lexicon_2021.to_csv('../../../../Code/Data/Test/Stock-Year/neg_lg_2021.csv', index=False)

In [13]:
a = list(set(list(neg_lexicon_2021['news_word'])))

a

['호전',
 '감소',
 '참패',
 '호황',
 '위축',
 '유발',
 '의심',
 '하락',
 '급감',
 '당혹',
 '과속',
 '연루',
 '저하',
 '운항',
 '급등',
 '강점',
 '파산보호',
 '끝내',
 '폭락',
 '폭로',
 '차량',
 '업황',
 '툴리눔',
 '보합',
 '실패',
 '강화',
 '약화',
 '위기',
 '상승',
 '부도',
 '비리',
 '보상',
 '증가',
 '등락',
 '논란',
 '약세',
 '성공',
 '무마',
 '경기',
 '불황',
 '반등',
 '악재',
 '견조',
 '좌절',
 '악화',
 '실린더헤드',
 '합선',
 '선박',
 '경색',
 '급락',
 '화물',
 '조처',
 '플러스',
 '침체',
 '주춤',
 '폭등',
 '엠앤티',
 '둔화',
 '손해배상',
 '불안',
 '역풍']

In [14]:
b = list(set(list(neg_lexicon_2021[neg_lexicon_2021['cosine_similarity']>=0.7]['news_word'])))

b

['침체', '성공', '실패', '약화', '의심', '상승']

In [15]:
c = list(set(list(neg_lexicon_2021[neg_lexicon_2021['cosine_similarity']>=0.65]['news_word'])))

c

['침체', '성공', '조처', '실패', '약화', '급락', '의심', '상승']